# **1. Membuat Constraint biar tidak ada double**

In [ ]:
// Player unik per full_name
CREATE CONSTRAINT player_fullname IF NOT EXISTS
FOR (p:Player)
REQUIRE p.full_name IS UNIQUE;

// Position unik per nama posisi
CREATE CONSTRAINT position_name IF NOT EXISTS
FOR (pos:Position)
REQUIRE pos.name IS UNIQUE;

// Country unik per nama negara
CREATE CONSTRAINT country_name IF NOT EXISTS
FOR (c:Country)
REQUIRE c.name IS UNIQUE;

// NationalTeam unik per nama tim nasional
CREATE CONSTRAINT nationalteam_name IF NOT EXISTS
FOR (t:NationalTeam)
REQUIRE t.name IS UNIQUE;

In [ ]:
// Mengecek constraint yang sudah dibuat

SHOW CONSTRAINTS;

# **2. IMPORT: Mapping SEMUA KOLOM ke (:Player)**

In [ ]:
LOAD CSV WITH HEADERS FROM 'file:///fifa_players.csv' AS row
WITH row
WHERE row.full_name IS NOT NULL

MERGE (p:Player {full_name: row.full_name})
SET
  // --- Identitas dasar ---
  p.name       = row.name,
  p.birth_date = row.birth_date,

  p.age        = CASE WHEN row.age = '' THEN null ELSE toInteger(row.age) END,
  p.height_cm  = CASE WHEN row.height_cm = '' THEN null ELSE toInteger(row.height_cm) END,
  p.weight_kgs = CASE WHEN row.weight_kgs = '' THEN null ELSE toInteger(row.weight_kgs) END,

  // Posisi & negara (string mentah, nanti dipakai bikin node Position/Country)
  p.positions   = row.positions,
  p.nationality = row.nationality,

  // Rating & value
  p.overall_rating = CASE WHEN row.overall_rating = '' THEN null ELSE toInteger(row.overall_rating) END,
  p.potential      = CASE WHEN row.potential = '' THEN null ELSE toInteger(row.potential) END,
  p.value_euro     = CASE WHEN row.value_euro = '' THEN null ELSE toInteger(row.value_euro) END,
  p.wage_euro      = CASE WHEN row.wage_euro = '' THEN null ELSE toInteger(row.wage_euro) END,

  // Kaki & reputasi
  p.preferred_foot          = row.preferred_foot,
  p.international_reputation = CASE
                                 WHEN row.`international_reputation(1-5)` = '' THEN null
                                 ELSE toInteger(row.`international_reputation(1-5)`)
                               END,
  p.weak_foot               = CASE
                                 WHEN row.`weak_foot(1-5)` = '' THEN null
                                 ELSE toInteger(row.`weak_foot(1-5)`)
                               END,
  p.skill_moves             = CASE
                                 WHEN row.`skill_moves(1-5)` = '' THEN null
                                 ELSE toInteger(row.`skill_moves(1-5)`)
                               END,
  p.body_type               = row.body_type,

  // Klausul & tim nasional
  p.release_clause_euro    = CASE
                               WHEN row.release_clause_euro = '' THEN null
                               ELSE toFloat(row.release_clause_euro)
                             END,
  p.national_team          = row.national_team,
  p.national_rating        = CASE
                               WHEN row.national_rating = '' THEN null
                               ELSE toInteger(row.national_rating)
                             END,
  p.national_team_position = row.national_team_position,
  p.national_jersey_number = CASE
                               WHEN row.national_jersey_number = '' THEN null
                               ELSE toInteger(row.national_jersey_number)
                             END,

  // --- Skill teknis & fisik ---
  p.crossing         = CASE WHEN row.crossing = '' THEN null ELSE toInteger(row.crossing) END,
  p.finishing        = CASE WHEN row.finishing = '' THEN null ELSE toInteger(row.finishing) END,
  p.heading_accuracy = CASE WHEN row.heading_accuracy = '' THEN null ELSE toInteger(row.heading_accuracy) END,
  p.short_passing    = CASE WHEN row.short_passing = '' THEN null ELSE toInteger(row.short_passing) END,
  p.volleys          = CASE WHEN row.volleys = '' THEN null ELSE toInteger(row.volleys) END,
  p.dribbling        = CASE WHEN row.dribbling = '' THEN null ELSE toInteger(row.dribbling) END,
  p.curve            = CASE WHEN row.curve = '' THEN null ELSE toInteger(row.curve) END,
  p.freekick_accuracy = CASE WHEN row.freekick_accuracy = '' THEN null ELSE toInteger(row.freekick_accuracy) END,
  p.long_passing      = CASE WHEN row.long_passing = '' THEN null ELSE toInteger(row.long_passing) END,
  p.ball_control      = CASE WHEN row.ball_control = '' THEN null ELSE toInteger(row.ball_control) END,

  p.acceleration      = CASE WHEN row.acceleration = '' THEN null ELSE toInteger(row.acceleration) END,
  p.sprint_speed      = CASE WHEN row.sprint_speed = '' THEN null ELSE toInteger(row.sprint_speed) END,
  p.agility           = CASE WHEN row.agility = '' THEN null ELSE toInteger(row.agility) END,
  p.reactions         = CASE WHEN row.reactions = '' THEN null ELSE toInteger(row.reactions) END,
  p.balance           = CASE WHEN row.balance = '' THEN null ELSE toInteger(row.balance) END,
  p.shot_power        = CASE WHEN row.shot_power = '' THEN null ELSE toInteger(row.shot_power) END,
  p.jumping           = CASE WHEN row.jumping = '' THEN null ELSE toInteger(row.jumping) END,
  p.stamina           = CASE WHEN row.stamina = '' THEN null ELSE toInteger(row.stamina) END,
  p.strength          = CASE WHEN row.strength = '' THEN null ELSE toInteger(row.strength) END,
  p.long_shots        = CASE WHEN row.long_shots = '' THEN null ELSE toInteger(row.long_shots) END,

  p.aggression        = CASE WHEN row.aggression = '' THEN null ELSE toInteger(row.aggression) END,
  p.interceptions     = CASE WHEN row.interceptions = '' THEN null ELSE toInteger(row.interceptions) END,
  p.positioning       = CASE WHEN row.positioning = '' THEN null ELSE toInteger(row.positioning) END,
  p.vision            = CASE WHEN row.vision = '' THEN null ELSE toInteger(row.vision) END,
  p.penalties         = CASE WHEN row.penalties = '' THEN null ELSE toInteger(row.penalties) END,
  p.composure         = CASE WHEN row.composure = '' THEN null ELSE toInteger(row.composure) END,
  p.marking           = CASE WHEN row.marking = '' THEN null ELSE toInteger(row.marking) END,
  p.standing_tackle   = CASE WHEN row.standing_tackle = '' THEN null ELSE toInteger(row.standing_tackle) END,
  p.sliding_tackle    = CASE WHEN row.sliding_tackle = '' THEN null ELSE toInteger(row.sliding_tackle) END;

In [ ]:
// Mengecek data player yang sudah masuk

MATCH (p:Player)
RETURN p.full_name, p.age, p.overall_rating, p.positions
LIMIT 10;

# **3. Bikin Node & Relasi: Position, Country, NationalTeam**

In [ ]:
// Player -> Position

LOAD CSV WITH HEADERS FROM 'file:///fifa_players.csv' AS row
WITH row
WHERE row.full_name IS NOT NULL AND row.positions IS NOT NULL AND row.positions <> ''

MATCH (p:Player {full_name: row.full_name})
UNWIND split(row.positions, ',') AS posName
WITH p, trim(posName) AS posName
MERGE (pos:Position {name: posName})
MERGE (p)-[:PLAYS_POSITION]->(pos);

// Melakukan pengecekan

MATCH (p:Player)-[:PLAYS_POSITION]->(pos:Position)
RETURN p.full_name, pos.name
LIMIT 10;

In [ ]:
// Player -> Country

LOAD CSV WITH HEADERS FROM 'file:///fifa_players.csv' AS row
WITH row
WHERE row.full_name IS NOT NULL AND row.nationality IS NOT NULL AND row.nationality <> ''

MATCH (p:Player {full_name: row.full_name})
MERGE (c:Country {name: row.nationality})
MERGE (p)-[:HAS_NATIONALITY]->(c);

// Melakukan pengecekan

MATCH (p:Player)-[:HAS_NATIONALITY]->(c:Country)
RETURN p.full_name, c.name
LIMIT 10;

In [ ]:
// Player -> National Team

LOAD CSV WITH HEADERS FROM 'file:///fifa_players.csv' AS row
WITH row
WHERE row.full_name IS NOT NULL
  AND row.national_team IS NOT NULL
  AND row.national_team <> ''

MATCH (p:Player {full_name: row.full_name})
MERGE (t:NationalTeam {name: row.national_team})
MERGE (p)-[:PLAYS_FOR_NATIONAL_TEAM]->(t);

// Melakukan pengecekan

MATCH (p:Player)-[:PLAYS_FOR_NATIONAL_TEAM]->(t:NationalTeam)
RETURN p.full_name, t.name
LIMIT 10;

# **4. Project GDS Graph: playersGraph**

Jangan lupa buat install plugin dulu di instance nya yaitu "Graph Data Science Library"

In [ ]:
CALL gds.graph.project(
  'playersGraph',
  {
    Player: {
      properties: [
        'age',
        'height_cm',
        'weight_kgs',
        'overall_rating',
        'potential',
        'crossing',
        'finishing',
        'heading_accuracy',
        'short_passing',
        'volleys',
        'dribbling',
        'curve',
        'freekick_accuracy',
        'long_passing',
        'ball_control',
        'acceleration',
        'sprint_speed',
        'agility',
        'reactions',
        'balance',
        'shot_power',
        'jumping',
        'stamina',
        'strength',
        'long_shots',
        'aggression',
        'interceptions',
        'positioning',
        'vision',
        'penalties',
        'composure',
        'marking',
        'standing_tackle',
        'sliding_tackle'
      ]
    },
    Position: {},
    Country: {},
    NationalTeam: {}
  },
  {
    PLAYS_POSITION: {orientation: 'UNDIRECTED'},
    HAS_NATIONALITY: {orientation: 'UNDIRECTED'},
    PLAYS_FOR_NATIONAL_TEAM: {orientation: 'UNDIRECTED'}
  }
);

In [ ]:
// Cek graph GDS sudah ke-project:

CALL gds.graph.list()
YIELD graphName, nodeCount, relationshipCount
RETURN graphName, nodeCount, relationshipCount;


# **5. Node Similarity → Relasi SIMILAR_TO**

In [ ]:
// Kalau mau membersihkan, pakai ini dulu

MATCH ()-[r:SIMILAR_TO]-()
DELETE r;

In [ ]:
CALL gds.nodeSimilarity.write(
  'playersGraph',
  {
    topK: 10,
    similarityCutoff: 0.6,       // misalnya, lebih ketat
    writeRelationshipType: 'SIMILAR_TO',
    writeProperty: 'score'
  }
)
YIELD nodesCompared, relationshipsWritten, similarityDistribution;

**topK: 10**
Untuk setiap pemain, hanya disimpan maksimal 10 pemain paling mirip.
Biar graph SIMILAR_TO tidak terlalu penuh.

**similarityCutoff: 0.6**
Hanya pasangan pemain dengan skor kemiripan ≥ 0.6 yang akan dibuatkan relasi.
Artinya: Kalau similarity-nya lemah (misal 0.2, 0.3) → diabaikan. Jadi yang tersisa benar-benar pasangan yang cukup mirip.

**writeRelationshipType: 'SIMILAR_TO'**
Menentukan nama relationship yang akan dibuat di database.
Hasilnya: (:Player)-[:SIMILAR_TO]->(:Player).

**writeProperty: 'score'**
Nilai kemiripan disimpan di property score pada relationship.
Misalnya:
(:Player {name: "Koulibaly"})-[:SIMILAR_TO {score: 0.93}]->(:Player {name: "Ba"}).


**CATATAN:** Faktor kemiripan antar pemain dilihat dari struktur dari graphnya, bukan statistik skill. Dua pemain dianggap mirip jika mereka memiliki hubungan yang sama, misalnya bermain pada posisi yang sama, berasal dari negara yang sama, atau berada dalam tim nasional yang sama. Jadi NodeSimilarity mengukur kesamaan pola tetangga antar node pada graph.

In [ ]:
// Cek

MATCH (p1:Player)-[r:SIMILAR_TO]->(p2:Player)
RETURN p1.full_name AS player1, p2.full_name AS player2, r.score
ORDER BY r.score DESC
LIMIT 10;

# **6. Project Graph Kemiripan: playersSimilarityGraph**

In [ ]:
CALL gds.graph.project(
  'playersSimilarityGraph',
  'Player',
  {
    SIMILAR_TO: {
      type: 'SIMILAR_TO',
      orientation: 'UNDIRECTED',
      properties: 'score'
    }
  }
);

In [ ]:
// Pengecekan

CALL gds.graph.list()
YIELD graphName, nodeCount, relationshipCount
RETURN graphName, nodeCount, relationshipCount;

# **7. PageRank (Centrality) di Graph Similarity**

In [ ]:
CALL gds.pageRank.write(
  'playersSimilarityGraph',
  {
    maxIterations: 20,
    dampingFactor: 0.85,
    writeProperty: 'pagerank'
  }
)
YIELD nodePropertiesWritten;

In [ ]:
// Pengecekan

MATCH (p:Player)
RETURN p.full_name, p.pagerank
ORDER BY p.pagerank DESC
LIMIT 10;


# **8. Community Detection (Louvain)**

In [ ]:
CALL gds.louvain.write(
  'playersSimilarityGraph',
  {
    writeProperty: 'communityId'
  }
)
YIELD communityCount, modularity, modularities;

In [ ]:
// Melihat beberapa komunitas

MATCH (p:Player)
RETURN p.communityId, count(*) AS jumlah_pemain
ORDER BY jumlah_pemain DESC
LIMIT 10;

In [ ]:
// Melihat komunitas lebih detail

MATCH (p:Player)
WHERE p.communityId = 10460   // ganti nomor community sesuai hasil
RETURN p.full_name, p.positions, p.overall_rating
LIMIT 20;

# **9. Node2Vec Embedding**

In [ ]:
CALL gds.node2vec.write(
  'playersSimilarityGraph',
  {
    embeddingDimension: 64,
    walkLength: 40,
    walksPerNode: 10,
    windowSize: 10,
    inOutFactor: 1.0,
    returnFactor: 1.0,
    writeProperty: 'embedding'
  }
)
YIELD nodePropertiesWritten;

**'playersSimilarityGraph'**
Graph yang dipakai: jaringan Player–SIMILAR_TO yang tadi kamu buat.

**embeddingDimension: 64**
Panjang vektor embedding = 64 angka per pemain.
Semakin besar → bisa representasi lebih kaya, tapi file lebih berat.

**walkLength: 40 & walksPerNode: 10**
Node2Vec melakukan random walk di graph:
Dari tiap pemain, jalan-jalan sepanjang 40 langkah
Diulang 10 kali per pemain
Tujuannya: “mengamati” lingkungan dan pola tetangga pemain di graph.

**windowSize: 10**
Ukuran konteks saat melatih embedding (mirip Word2Vec di NLP):
Node yang sering muncul berdekatan dalam random walk → embedding-nya mirip.

**inOutFactor, returnFactor (1.0 = netral)**
Mengontrol apakah random walk lebih BFS-like (lokal) atau DFS-like (global).

In [ ]:
MATCH (p:Player)
WHERE p.embedding IS NOT NULL
RETURN p.full_name, p.embedding[0..5] AS embedding_example
LIMIT 5;

# **10. Siap Export ke ML**

In [ ]:
MATCH (p:Player)
WHERE p.embedding IS NOT NULL AND p.positions IS NOT NULL
RETURN
  p.full_name       AS full_name,
  p.positions       AS positions,
  p.overall_rating  AS overall_rating,
  p.age             AS age,
  p.acceleration    AS acceleration,
  p.sprint_speed    AS sprint_speed,
  p.dribbling       AS dribbling,
  p.short_passing   AS short_passing,
  p.finishing       AS finishing,
  p.stamina         AS stamina,
  p.strength        AS strength,
  p.embedding       AS embedding;